**This is the code of RC-ESN to conduct prediction on the multi-scale Lorenz 96.The two components of the RC-ESN are an input-to-reservoir
layer with weight matrix Win, and a reservoir-to-output layer with weight matrix Wout.  A and Win are chosen randomly and are fixed i.e., they do not change during training or testing. During training, only the
weights of the output-to-reservoir layer (Wout) are updated.**


In [ ]:
#importing all the required libraries.
import numpy as np
import scipy.sparse as sparse
from scipy.sparse import linalg
import pandas as pd


In [ ]:
#Defining the global variables
#This will change the initial condition used. Currently it starts from the first value
shift_k = 0

approx_res_size = 5000

#model parameters
model_params = {'tau': 0.25,      
                'nstep': 1000,
                'N': 8,
                'd': 22}

#reservior parameters
res_params = {'radius':0.1,
             'degree': 3,
             'sigma': 0.5,
             'train_length': 100000,
             'N': int(np.floor(approx_res_size/model_params['N']) * model_params['N']),
             'num_inputs': model_params['N'],
             'predict_length': 10000,
             'beta': 0.0001
              }


In [ ]:
# The ESN functions for training

#Generating Reservoir A such data spectral radius as to be less than unity
def generate_reservoir(size,radius,degree):
    sparsity = degree/float(size);
    A = sparse.rand(size,size,density=sparsity).todense()             #sparsely connected neurons
    vals = np.linalg.eigvals(A)
    e = np.max(np.abs(vals))                                          # e is largest eigen value
    A = (A/e) * radius                                                
    return A

#Computing state matrix of reservoir layer
def reservoir_layer(A, Win, input, res_params):
    states = np.zeros((res_params['N'],res_params['train_length']))   
    for i in range(res_params['train_length']-1):
        states[:,i+1] = np.tanh(np.dot(A,states[:,i]) + np.dot(Win,input[:,i])) #states are calculated using r(t + ∆t) = tanh (Ar(t) + WinX(t))
    return states

#training the reservoir
def train_reservoir(res_params, data):
    A = generate_reservoir(res_params['N'], res_params['radius'], res_params['degree'])   #calling generate_reservoir function
    q = int(res_params['N']/res_params['num_inputs'])                                     #No. of neuron in a layer of reservoir
    Win = np.zeros((res_params['N'],res_params['num_inputs']))
    for i in range(res_params['num_inputs']):
        np.random.seed(seed=i)
        Win[i*q: (i+1)*q,i] = res_params['sigma'] * (-1 + 2 * np.random.rand(1,q)[0])    #weight matrix win whose values are drawn from a uniform random distribution on[-1,1]
        
    states = reservoir_layer(A, Win, data, res_params)                                  #calling reservoir_layer function
    Wout = train(res_params, states, data)                                              #calling train function
    x = states[:,-1]
    return x, Wout, A, Win


#Computing trainable matrix Wout
def train(res_params,states,data):
    beta = res_params['beta']
    idenmat = beta * sparse.identity(res_params['N'])    #*
    states2 = states.copy()

    #nonlinear combinations of the columns of the reservoir state matrix using algorithm T2
    for j in range(2,np.shape(states2)[0]-2):                           
        if (np.mod(j,2)==0):
            states2[j,:] = (states[j-1,:]*states[j-2,:]).copy()            # using Algorithm T2 ,if j > 1 is odd
    U = np.dot(states2,states2.transpose()) + idenmat                      #
    Uinv = np.linalg.inv(U)                                                
    Wout = np.dot(Uinv,np.dot(states2,data.transpose()))                   
    return Wout.transpose()                                               #reservior-to-output layer weight matrix wout


#Wout is marching forward in time during prediction(testing)
def predict(A, Win, res_params, x, Wout):
    output = np.zeros((res_params['num_inputs'],res_params['predict_length']))
    for i in range(res_params['predict_length']):
        x_aug = x.copy()
        for j in range(2,np.shape(x_aug)[0]-2):
            if (np.mod(j,2)==0):                                          #using Algorithm T2 ,if j > 1 is odd
                x_aug[j] = (x[j-1]*x[j-2]).copy()
        out = np.squeeze(np.asarray(np.dot(Wout,x_aug)))                  #using prediction process equation
        output[:,i] = out
        x1 = np.tanh(np.dot(A,x) + np.dot(Win,out))                       #states are getting calculated for further prediction
        x = np.squeeze(np.asarray(x1))
    return output, x


In [ ]:
#loading Lorenz data
dataf = pd.read_csv("lorenz_data.csv",header=None)
data = np.transpose(np.array(dataf))

# Train reservoir
x,Wout,A,Win = train_reservoir(res_params,data[:,shift_k:shift_k+res_params['train_length']])

# Prediction
output, _ = predict(A, Win,res_params,x,Wout)
np.save('Expansion_2step_back'+'R_size_train_'+str(res_params['train_length'])+'_Rd_'+str(res_params['radius'])+'_Shift_'+str(shift_k)+'.npy',output)